# Попытка сделать класс

In [59]:
import os
import pandas as pd
from tqdm import tqdm
import torch
import pyarrow.parquet as pq
import dask.dataframe as dd
import spacy
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, BertTokenizerFast, BertForSequenceClassification, BertConfig
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter
from torch.utils.data import DataLoader, Dataset as TorchDataset
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import hdbscan
from scipy.spatial.distance import pdist, squareform
import logging
import re
from joblib import Parallel, delayed


class ReviewsKeywords:
    def __init__(self, csv_path, model_path, spacy_model="ru_core_news_lg"):
        self.csv_path = csv_path
        self.model_path = model_path

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if self.device == "cuda":
            import cudf.pandas  # Импортирование cuDF и активация его использования
            cudf.pandas.install()
        os.environ["TOKENIZERS_PARALLELISM"] = "true"  # Включаем параллелизм токенизатора для ускорения
        self.tokenizer_my = BertTokenizerFast.from_pretrained(self.model_path)
         # Загрузка модели для классификации
        self.classification_model = BertForSequenceClassification.from_pretrained(self.model_path).to(self.device)
        # Загрузка базовой модели для получения эмбеддингов
        self.embedding_model = AutoModel.from_pretrained(self.model_path).to(self.device)
        
        # Загрузка модели и токенайзера от Сбербанка
        self.tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/sbert_large_nlu_ru')
        self.embedding_model = AutoModel.from_pretrained('sberbank-ai/sbert_large_nlu_ru').to(self.device)
        
        spacy.prefer_gpu()
        self.nlp = spacy.load(spacy_model, disable=["ner", "tagger", "attribute_ruler", "lemmatizer"])
        
        self.df = pd.read_csv(self.csv_path, nrows=1000)

    @staticmethod
    def clean_text(text):
        text = re.sub(r'[\n\r\t]+|\s{2,}', ' ', text)
        text = re.sub(r'(?<!\.)\s*\.\s*|\s*\.\s*(?!\.)', '. ', text)
        return text.strip().rstrip('.')

    def split_reviews_into_sentences(self, batch):
        cleaned_texts = [self.clean_text(text) for text in batch['corrected_text']]
        docs = list(self.nlp.pipe(cleaned_texts, batch_size=64))
        batch['sentences'] = [[sent.text for sent in doc.sents] for doc in docs]
        return batch

    def process_reviews(self):
        dataset = Dataset.from_pandas(self.df)
        dataset = dataset.map(self.split_reviews_into_sentences, batched=True, batch_size=32)
        self.df = dataset.to_pandas()
        df_exploded = self.df.explode('sentences').reset_index(drop=True)
        df_exploded = df_exploded.drop(columns=[col for col in df_exploded.columns if col.startswith('__index_level_')])
        return Dataset.from_pandas(df_exploded)

    def compute_sentence_embeddings(self, sentences):
        sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
        if not sentences:
            raise ValueError("Input contains no valid strings.")
        inputs = self.tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.embedding_model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    def compute_embeddings_after_explode(self, batch):
        sentences = batch['sentences']
        valid_sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
        if not valid_sentences:
            batch['sentence_embeddings'] = [[]] * len(sentences)
            return batch
        embeddings = self.compute_sentence_embeddings(valid_sentences)
        embeddings = embeddings.astype(np.float32)
        final_embeddings = []
        embed_idx = 0
        for sentence in sentences:
            if isinstance(sentence, str):
                final_embeddings.append(embeddings[embed_idx])
                embed_idx += 1
            else:
                final_embeddings.append(np.zeros(embeddings.shape[1], dtype=np.float32))
        batch['sentence_embeddings'] = final_embeddings
        return batch

    def apply_embeddings(self, dataset_exploded):
        return dataset_exploded.map(self.compute_embeddings_after_explode, batched=True, batch_size=128)

    def extract_key_thought(self, cluster_sentences):
        sentences = cluster_sentences.split(" | ")
        embeddings = self.compute_sentence_embeddings(sentences)
        centroid = np.mean(embeddings, axis=0)
        similarities = cosine_similarity(embeddings, [centroid])
        key_sentence_index = np.argmax(similarities)
        return sentences[key_sentence_index]

    def count_words(self, cluster_sentences):
        words = cluster_sentences.split()
        return len(words)

    def recluster_large_cluster(self, cluster_sentences, eps=0.1, min_samples=2):
        sentences = cluster_sentences.split(" | ")
        embeddings = self.compute_sentence_embeddings(sentences)
        re_clustering = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine").fit(embeddings)
        re_cluster_dict = {}
        for idx, label in enumerate(re_clustering.labels_):
            if label == -1:
                continue
            label_str = str(label)
            if label_str not in re_cluster_dict:
                re_cluster_dict[label_str] = []
            re_cluster_dict[label_str].append(sentences[idx])
        return [" | ".join(cluster) for cluster in re_cluster_dict.values()]

    def recursive_clustering(self, cluster_sentences, threshold, eps=0.22, min_samples=3, min_eps=0.02):
        current_eps = eps
        current_min_samples = min_samples
        new_clusters = [cluster_sentences]
        while True:
            next_clusters = []
            reclustered_any = False
            for cluster in new_clusters:
                if self.count_words(cluster) > threshold:
                    while current_eps >= min_eps:
                        reclustered = self.recluster_large_cluster(cluster, eps=current_eps, min_samples=current_min_samples)
                        if len(reclustered) > 1:
                            next_clusters.extend(reclustered)
                            reclustered_any = True
                            break
                        else:
                            if current_eps > min_eps:
                                current_eps -= 0.05
                    if len(reclustered) == 1:
                        next_clusters.append(cluster)
                else:
                    next_clusters.append(cluster)
            new_clusters = next_clusters
            if not reclustered_any:
                break
        return new_clusters

    def generate_predictions(self, dataset_exploded):
        tokenizer = self.tokenizer_my
        model = self.classification_model
        if self.device == torch.device("cuda"):
            model = model.half()

        reviews = dataset_exploded["sentences"]
        reviews = [str(review) for review in reviews if isinstance(review, str) and review.strip()]

        class ReviewDataset(TorchDataset):
            def __init__(self, reviews, tokenizer, max_len=128):
                self.reviews = reviews
                self.tokenizer = tokenizer
                self.max_len = max_len

            def __len__(self):
                return len(self.reviews)

            def __getitem__(self, idx):
                review = self.reviews[idx]
                encoding = self.tokenizer.encode_plus(
                    review,
                    add_special_tokens=True,
                    max_length=self.max_len,
                    return_token_type_ids=False,
                    padding='max_length',
                    truncation=True,
                    return_attention_mask=True,
                    return_tensors='pt'
                )
                return {key: val.flatten() for key, val in encoding.items()}

        dataset = ReviewDataset(reviews, tokenizer)
        batch_size = 32
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        predictions = []

        from torch.cuda.amp import autocast

        for batch in tqdm(dataloader, desc="Предсказание отзывов"):
            batch = {key: val.to(self.device) for key, val in batch.items()}
            
            with torch.no_grad():
                with autocast():  # Используем смешанную точность
                    outputs = model(**batch)
                    logits = outputs[0] if isinstance(outputs, tuple) else outputs.logits
                    probabilities = torch.softmax(logits, dim=-1)
                    batch_predictions = (probabilities[:, 1] > 0.7).cpu().numpy()  # Используем порог 0.7
                    predictions.extend(batch_predictions)

        if len(predictions) != len(dataset_exploded):
            print(f"Warning: Length of predictions ({len(predictions)}) does not match length of index ({len(dataset_exploded)})")
            if len(predictions) < len(dataset_exploded):
                missing_count = len(dataset_exploded) - len(predictions)
                predictions.extend([0] * missing_count)
            elif len(predictions) > len(dataset_exploded):
                predictions = predictions[:len(dataset_exploded)]
        dataset_exploded = dataset_exploded.add_column("predictions", predictions)
        return dataset_exploded

    def process_group(self, category_name, product_name, group):
        all_sentences = group['sentences'].tolist()
        if not all_sentences:
            return pd.DataFrame()

        try:
            all_embeddings = self.compute_sentence_embeddings(all_sentences)
        except ValueError as e:
            print(f"Error in computing embeddings for product {product_name}: {e}")
            return pd.DataFrame()

        distance_matrix = squareform(pdist(all_embeddings, metric='cosine'))
        clustering = hdbscan.HDBSCAN(min_samples=3, metric='precomputed').fit(distance_matrix)

        cluster_dict = {}
        for idx, label in enumerate(clustering.labels_):
            if label == -1:
                continue
            label_str = str(label)
            if label_str not in cluster_dict:
                cluster_dict[label_str] = set()
            cluster_dict[label_str].add(all_sentences[idx])

        clusters = [" | ".join(sentences) for sentences in cluster_dict.values()]

        if not clusters:
            return pd.DataFrame()

        group['binary_rating'] = group['review_rating'].apply(lambda x: 1 if x in [4, 5] else 0)
        avg_rating = group['binary_rating'].mean()
        rating_category = 'positive' if avg_rating > 0.7 else 'neutral'
        rating_category = 'neutral' if avg_rating > 0.5 else 'negative'

        threshold = self.determine_threshold(clusters)

        final_clusters = []
        for cluster in clusters:
            if self.count_words(cluster) > threshold:
                final_clusters.extend(self.recursive_clustering(cluster, threshold))
            else:
                final_clusters.append(cluster)

        # Обеспечение минимального количества кластеров
        final_clusters = self.ensure_minimum_clusters(final_clusters, threshold)

        df_exploded_sorted = pd.DataFrame({
            'category': category_name,
            'product': product_name,
            'avg_rating': avg_rating,
            'rating_category': rating_category,
            'cluster_sentences': final_clusters
        })
        df_exploded_sorted['word_count'] = df_exploded_sorted['cluster_sentences'].apply(self.count_words)
        df_exploded_sorted['key_thought'] = df_exploded_sorted['cluster_sentences'].apply(self.extract_key_thought)
        df_exploded_sorted = df_exploded_sorted.sort_values(by='word_count', ascending=False)

        return df_exploded_sorted

    def determine_threshold(self, clusters):
        if len(clusters) == 1:
            cluster_word_count = self.count_words(clusters[0])
            if cluster_word_count > 20:
                return cluster_word_count / 2
            return cluster_word_count
        return np.min([np.mean([self.count_words(cluster) for cluster in clusters]) * 1.5, 250])

    def ensure_minimum_clusters(self, final_clusters, threshold):
        while len(final_clusters) < 3 and any(self.count_words(cluster) > threshold for cluster in final_clusters):
            largest_cluster = max(final_clusters, key=self.count_words)
            final_clusters.remove(largest_cluster)
            new_clusters = self.recursive_clustering(largest_cluster, threshold)
            if len(new_clusters) <= 1:
                final_clusters.append(largest_cluster)
                break
            final_clusters.extend(new_clusters)
        return final_clusters
    
    def cluster_reviews(self, dataset_exploded):
        # Фильтрация на основе предсказаний
        dataset_filtered = dataset_exploded.filter(lambda x: x['predictions'] == 1)
        
        # Преобразование в pandas DataFrame для группировки
        df_filtered = dataset_filtered.to_pandas()
        grouped = df_filtered.groupby(['category', 'product'])

        results = []
        
        # Последовательная обработка без параллелизма
        for (category_name, product_name), group in tqdm(grouped, desc="Processing categories and products"):
            result_df = self.process_group(category_name, product_name, group)
            if not result_df.empty:
                results.append(result_df)

        if results:  # Проверяем, что список results не пуст
            final_result = pd.concat(results, ignore_index=True)
            final_result = final_result[((final_result['word_count'] > 10) & (final_result['key_thought'].str.len() > 5))]
            final_result.to_csv("./reviews_keywords/feedbackfueltest.csv")
        else:
            print("No valid results to concatenate. Returning an empty DataFrame.")
            final_result = pd.DataFrame()  # Возвращаем пустой DataFrame, если нет данных для объединения
        
        return final_result

    def run(self):
        dataset_exploded = self.process_reviews()
        dataset_exploded = self.apply_embeddings(dataset_exploded)
        dataset_exploded = self.generate_predictions(dataset_exploded)
        result = self.cluster_reviews(dataset_exploded)
        return result


reviews_keywords = ReviewsKeywords(csv_path="./reviews_keywords/wildberries_reviews.csv",
                                    model_path='./reviews_keywords/fine_tuned_model')
final_result = reviews_keywords.run()
final_result.head()

ValueError: could not determine the shape of object type 'torch.storage.UntypedStorage'

In [ ]:
final_result

,category,product,avg_rating,rating_category,cluster_sentences,word_count,key_thought
0,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,0.819588,neutral,"Переднее колесо закрылось в снегу, подложили п...",40,"Переднее колесо закрылось в снегу, подложили п..."
1,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,0.819588,neutral,На вид крепкие. | Вроде прочные. | На вид проч...,12,На вид крепкие.
3,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,0.842975,neutral,В деле не пробовал | В деле пока не пробовала....,37,В деле не пробовал
4,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,0.704545,neutral,"Рекомендую, буду брать еще | Закажу | мыМыочно...",20,Буду заказывать ещё.
5,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,0.704545,neutral,Еле пахнет. | Он даже не пахнет. | Пахнет каки...,13,Еле пахнет.
6,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,0.704545,neutral,🔥🔥🔥🔥🔥🔥 запах. | Запах огонь) | Запах огонь!!!!...,11,Запах 🔥!!!
7,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,0.853933,neutral,Убирает ржавчину хорошо через 10-20 минут | Со...,76,Ржавчину убирает отлично.
8,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,0.853933,neutral,"Ржавчина уже хорошо въелась, пришлось нескольк...",38,"Ржавчина уже хорошо въелась, пришлось нескольк..."
9,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,0.853933,neutral,"Фото «до» к сожалению не сделала, только «посл...",24,"Фото «до» к сожалению не сделала, только «после»"
10,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,0.853933,neutral,В деле пока не пробовала. | В деле не пробовал...,14,В деле не пробовал


In [ ]:
final_result

,category,product,avg_rating,rating_category,cluster_sentences,word_count,key_thought
0,/Спорт/Страйкбол и пейнтбол/Аксессуары,karbi / Рюкзак тактический туристический - кар...,0.797101,neutral,"Много доп карманов, чехол от дождя, прорезинен...",203,"Рюкзак вместительный, прочный, есть защитный ч..."
1,/Спорт/Страйкбол и пейнтбол/Аксессуары,karbi / Рюкзак тактический туристический - кар...,0.797101,neutral,"В подарок шёл компас,, налобныйфонарь,, ножане...",69,"В подарок положили фонарик налобный, компас и ..."


# Этап 1

In [50]:
import cudf.pandas  # Импортирование cuDF и активация его использования
cudf.pandas.install()  # Установка cuDF как основного интерфейса для pandas
import os
import pandas as pd
from tqdm import tqdm
import torch
import pyarrow.parquet as pq
import dask.dataframe as dd

# # Чтение Parquet-файла с использованием pyarrow
# table = pq.read_table('./reviews_keywords/wildberries_reviews_corrected.parquet')

# # Преобразование в pandas DataFrame
# df_pandas = table.to_pandas()

# # Преобразование pandas DataFrame в Dask DataFrame
# df_dask = dd.from_pandas(df_pandas, npartitions=100)  # Укажите количество нужных вам частей
# df_pandas = None
# table = None
# import gc
# gc.collect()
# df_dask

In [51]:
result = pd.read_csv("./reviews_keywords/wildberries_reviews.csv", nrows=1000)
result.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Unnamed: 0        1000 non-null   int64
 1   review_full_text  1000 non-null   object
 2   review_rating     1000 non-null   int64
 3   product           1000 non-null   object
 4   category          1000 non-null   object
 5   url               1000 non-null   object
 6   corrected_text    1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 540.9+ KB


In [52]:
# Оставляем только по 5 записей для каждого уникального значения в столбце 'product'
# result_limited = result.groupby('product').apply(lambda x: x.iloc[5:8]).reset_index(drop=True)
result_limited = result


In [79]:
import spacy
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, BertTokenizerFast
import torch
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, BertConfig
import hdbscan
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm

# Загрузка дообученной модели и токенизатора
# Загружаем конфигурацию модели


# Загрузка модели и токенайзера от Сбербанка
tokenizer = BertTokenizerFast.from_pretrained("./reviews_keywords/fine_tuned_model")
model = AutoModel.from_pretrained("./reviews_keywords/fine_tuned_model").to(device)
# Инициализируем модель с конфигурацией
config = BertConfig.from_pretrained('./reviews_keywords/fine_tuned_model', output_hidden_states=True)
model_classification = BertForSequenceClassification.from_pretrained('./reviews_keywords/fine_tuned_model', config=config).to(device)

        # self.tokenizer_my = BertTokenizerFast.from_pretrained(self.model_path)
        #  # Загрузка модели для классификации
        # self.classification_model = BertForSequenceClassification.from_pretrained(self.model_path).to(self.device)
        # # Загрузка базовой модели для получения эмбеддингов
        # self.embedding_model = AutoModel.from_pretrained(self.model_path).to(self.device)

spacy.prefer_gpu()
# Загрузка и настройка модели SpaCy
nlp = spacy.load("ru_core_news_lg", disable=["ner", "tagger", "attribute_ruler", "lemmatizer"])

df = result_limited

# Преобразование pandas DataFrame в Hugging Face Dataset
dataset = Dataset.from_pandas(df)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [80]:
import re

def clean_text(text):
    text = re.sub(r'[\n\r\t]+|\s{2,}', ' ', text)  # Объединяем шаги для замены пробелов
    text = re.sub(r'(?<!\.)\s*\.\s*|\s*\.\s*(?!\.)', '. ', text)  # Оптимизация замены точки
    return text.strip().rstrip('.')

def split_by_syntax(doc, max_length=100):
    """
    Разделяет длинные предложения на более короткие, основываясь на синтаксическом анализе.
    """
    split_sentences = []
    current_chunk = []
    if len(doc.text) < 100:
        return [doc.text]
    for token in doc:
        current_chunk.append(token.text)
        
        # Проверка, можно ли завершить предложение
        if token.dep_ in ('punct', 'cc', 'conj') or len(' '.join(current_chunk)) > max_length:
            if len(' '.join(current_chunk)) > max_length:
                split_sentences.append(' '.join(current_chunk[:-1]))
                current_chunk = [current_chunk[-1]]
            else:
                split_sentences.append(' '.join(current_chunk))
                current_chunk = []

    # Добавление последнего оставшегося фрагмента
    if current_chunk:
        split_sentences.append(' '.join(current_chunk))

    return split_sentences

def split_reviews_into_sentences(batch):
    # Очистка текстов
    cleaned_texts = [clean_text(text) for text in batch['corrected_text']]
    
    # Обработка текстов с помощью nlp.pipe с указанием batch_size
    docs = list(nlp.pipe(cleaned_texts, batch_size=64))

    # Извлечение предложений и их дальнейшая обработка
    processed_sentences = []
    for doc in docs:
        sentences = []
        for sent in doc.sents:
            # Разделяем длинные предложения по синтаксису
            split_sentences = split_by_syntax(sent, max_length=100)
            sentences.extend(split_sentences)
        processed_sentences.append(sentences)
    
    batch['sentences'] = processed_sentences
    
    return batch

dataset = dataset.map(split_reviews_into_sentences, batched=True, batch_size=32)

# Преобразуем Dataset обратно в pandas DataFrame
df = dataset.to_pandas()

# Выполним explode по столбцу с предложениями
df_exploded = df.explode('sentences').reset_index(drop=True)

# Удаляем лишние столбцы, которые появились после explode
df_exploded = df_exploded.drop(columns=[col for col in df_exploded.columns if col.startswith('__index_level_')])

# Преобразуем DataFrame обратно в Hugging Face Dataset
dataset_exploded = Dataset.from_pandas(df_exploded)

from torch.cuda.amp import autocast

def compute_sentence_embeddings(sentences):
    # Фильтруем список, оставляя только строки
    sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
    
    if not sentences:
        raise ValueError("Input contains no valid strings.")

    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    
    with torch.no_grad():
        with autocast():  # Используем mixed precision для ускорения
            outputs = model(**inputs)
    
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

def compute_embeddings_after_explode(batch):
    sentences = batch['sentences']

    # Проверяем, что все элементы в batch являются строками
    valid_sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
    
    if not valid_sentences:
        batch['sentence_embeddings'] = [[]] * len(sentences)  # Если нет валидных предложений, возвращаем пустые эмбеддинги
        return batch

    embeddings = compute_sentence_embeddings(valid_sentences)

    # Приведение эмбеддингов к типу float32 для консистентности
    embeddings = embeddings.astype(np.float32)

    # Проверяем, что количество эмбеддингов совпадает с количеством предложений
    if len(embeddings) != len(valid_sentences):
        raise ValueError("Количество эмбеддингов не совпадает с количеством предложений.")
    
    # Если количество предложений после фильтрации не совпадает с исходным, корректируем выходные данные
    final_embeddings = []
    embed_idx = 0
    for sentence in sentences:
        if isinstance(sentence, str):
            final_embeddings.append(embeddings[embed_idx])
            embed_idx += 1
        else:
            final_embeddings.append(np.zeros(embeddings.shape[1], dtype=np.float32))  # Добавляем нулевые эмбеддинги для невалидных предложений

    batch['sentence_embeddings'] = final_embeddings
    return batch

# Применение функции
dataset = dataset_exploded.map(compute_embeddings_after_explode, batched=True, batch_size=128)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3243 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [81]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, BertForSequenceClassification
from tqdm import tqdm
import os
import os
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, BertConfig
import nltk
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm
import logging
import hdbscan  # HDBSCAN для более стабильной кластеризации с поддержкой кастомных метрик
from scipy.spatial.distance import pdist, squareform

In [82]:
# Определение устройства (GPU или CPU)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Перевод модели в режим FP16, если это возможно
if use_cuda:
    model_classification = model_classification.half()

# Пример данных (замените на реальные данные)
reviews = dataset_exploded["sentences"]

# Очистка данных от некорректных значений
reviews = [str(review) for review in reviews if isinstance(review, str) and review.strip()]

# Создание кастомного Dataset для обработки отзывов
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_len=128):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {key: val.flatten() for key, val in encoding.items()}

# Создаем датасет и DataLoader
dataset = ReviewDataset(reviews, tokenizer)
batch_size = 32  # Размер батча можно изменить в зависимости от объема доступной памяти GPU
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Получение предсказаний с отображением прогресса
predictions = []

from torch.cuda.amp import autocast  # Импортируем autocast для смешанной точности

for batch in tqdm(dataloader, desc="Предсказание отзывов"):
    batch = {key: val.to(device) for key, val in batch.items()}
    
    with torch.no_grad():
        with autocast():  # Используем смешанную точность
            outputs = model_classification(**batch)
            logits = outputs[0] if isinstance(outputs, tuple) else outputs.logits
            probabilities = torch.softmax(logits, dim=-1)
            batch_predictions = (probabilities[:, 1] > 0.7).cpu().numpy()  # Используем порог 0.7
            predictions.extend(batch_predictions)

# Преобразование в DataFrame, если это еще не сделано
if not isinstance(dataset_exploded, pd.DataFrame):
    dataset_exploded = pd.DataFrame(dataset_exploded)

# Проверка и обработка несоответствия длины
if len(predictions) != len(dataset_exploded):
    print(f"Warning: Length of predictions ({len(predictions)}) does not match length of index ({len(dataset_exploded)})")
    
    # Пример: Заполнение недостающих значений
    if len(predictions) < len(dataset_exploded):
        missing_count = len(dataset_exploded) - len(predictions)
        predictions.extend([0] * missing_count)  # Добавляем нули в случае недостатка предсказаний

    elif len(predictions) > len(dataset_exploded):
        predictions = predictions[:len(dataset_exploded)]  # Обрезаем список предсказаний

# Присоединение предсказаний к датасету
dataset_exploded['predictions'] = predictions
dataset_exploded.head()



Предсказание отзывов:   0%|                                                                                                                                                                       | 0/102 [00:00<?, ?it/s]

Предсказание отзывов: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 102/102 [00:16<00:00,  6.16it/s]


,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_text,sentences,__index_level_0__,predictions
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.,Работает хорошо,0,False
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","Пришло быстро, все целое на вид.",1,False
2,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать,2,True
3,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала, установка ...",3,True
4,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая.,4,True


In [83]:
# Группировка по 'corrected_text' и объединение предложений в 'sentences'
# df_filtered_reviews = dataset_exploded[dataset_exploded["predictions"] == 1].groupby(["review_full_text", "review_rating", "product", "category", "url", "corrected_text"])['sentences'].apply(lambda x: " ".join(x)).reset_index()
df_filtered_reviews = dataset_exploded[dataset_exploded["predictions"] == 1][["review_full_text", "review_rating", "product", "category", "url", "corrected_text", 'sentences']]
df_filtered_reviews

,review_full_text,review_rating,product,category,url,corrected_text,sentences
2,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала, установка ..."
4,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая.
5,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Но в инструкции ни слова про сборку и креплени...
7,Муж ещё не пробовал,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Муж ещё не пробовал,Муж ещё не пробовал
...,...,...,...,...,...,...,...
3228,Средство рабочее надо брать,5,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,Средство рабочее надо брать,Средство рабочее надо брать
3235,Хорошо убирает ржавчину,5,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,Хорошо убирает ржавчину,Хорошо убирает ржавчину
3239,"так не о чем, эффекта почти нет, бестолковая х...",3,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,"Так не о чем, эффекта почти нет, бестолковая х...","Так не о чем, эффекта почти нет, бестолковая х..."
3240,Посмотрим что и как будет,5,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,Посмотрим что и как будет,Посмотрим что и как будет


In [84]:
df_filtered_reviews["corrected_text"] = df_filtered_reviews["corrected_text"].astype(str)
df_sorted_by_length = df_filtered_reviews.sort_values(by="corrected_text", key=lambda x: x.str.len(), ascending=False)
df_sorted_by_length.iloc[0]["corrected_text"]

'Не первая лебедка из поднебесной. Перед установкой разобрал, посмотрел реально ли влагозащищенная - реально, есть сальники! Если не топить ее в воде полностью, то влаги в ней не будет, от брызг, дождя, обильного поливания водой сверху защиты будет достаточно. Смазки в редукторе было не мало, но все равно добавил еще. Соленоиды в блоке управления самые простые, для рыбаков и охотников пойдет (если пользоваться ей редко). Блок соленоидов не влагозащищен - прячьте в подкопаться его подальше от влаги.  Провода родные заменил т.к. на мой взгляд тонковаты были. Оба пульта управления самые простые. Вероятно при сборке попутали провода смотки/размотки местами т.к. оба пульта при нажатии на кнопку размотки наоборот наматывают трос - не критично.\nВ целом за такую стоимость хороший вариант. Посмотрим как будет работать, пока проверил только после установки на работоспособность, в "бою" испытания не проходила.'

In [85]:
# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_lg")

# Установка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
from loguru import logger
logger.add('./reviews_keywords/clustering.log', encoding='utf-8')

3

In [87]:
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset

# Кастомный Dataset для DataLoader
class SentencesDataset(Dataset):
    def __init__(self, sentences, tokenizer, max_length=128):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        # Токенизация строки
        encoded = self.tokenizer(
            self.sentences[idx],
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=self.max_length
        )
        # Возвращаем input_ids и attention_mask
        return encoded['input_ids'].squeeze(0), encoded['attention_mask'].squeeze(0)

# Функция для обработки батчей и получения эмбеддингов
def get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32):
    dataset = SentencesDataset(sentences, tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    all_embeddings = []
    model.eval()
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            all_embeddings.extend(embeddings)
    
    return np.array(all_embeddings)


# Перевод модели в режим FP16, если это возможно
if torch.cuda.is_available():
    model_classification = model_classification.half()

def find_centroid(embeddings):
    return np.mean(embeddings, axis=0)

def extract_key_thought(sentences):
    embeddings = get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32)
    centroid = find_centroid(embeddings)
    similarities = cosine_similarity(embeddings, [centroid])
    key_sentence_index = np.argmax(similarities)
    return sentences[key_sentence_index]

def count_words(cluster_sentences):
    words = " ".join(cluster_sentences).split()
    return len(words)


In [88]:
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

# Загрузка модели для резюмирования на русском языке (если она доступна)
summarizer = pipeline("summarization", model="IlyaGusev/rugpt3medium_sum_gazeta")

# Функции для различных методов извлечения ключевой мысли
def extract_key_thought_method_1(sentences):
    embeddings = get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32)
    centroid = find_centroid(embeddings)
    similarities = cosine_similarity(embeddings, [centroid])
    key_sentence_index = np.argmax(similarities)
    return sentences[key_sentence_index]

def extract_key_thought_method_2(sentences):
    embeddings = get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32)
    centroid = find_centroid(embeddings)
    similarities = cosine_similarity(embeddings, [centroid])
    top_indices = np.argsort(similarities, axis=0)[-3:]  # Выбираем три наиболее близких предложения
    key_thought = " ".join([sentences[idx] for idx in top_indices.flatten()])
    return key_thought

def extract_key_thought_method_4(sentences):
    embeddings = get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32)
    centroid = find_centroid(embeddings)
    similarities = cosine_similarity(embeddings, [centroid])
    key_sentence_index = np.argmax(similarities)
    key_sentence = sentences[key_sentence_index]

    # Проверяем, не пустое ли предложение и не состоит ли оно только из стоп-слов
    if len(key_sentence.strip()) == 0:
        return "Предложение пустое или состоит только из стоп-слов."

    vectorizer = CountVectorizer(max_features=5, stop_words=stop_words)  # Используем русские стоп-слова
    X = vectorizer.fit_transform([key_sentence])
    
    if X.shape[1] == 0:
        return f"Основной мысли нет"  # Возвращаем исходное предложение

    key_phrases = " ".join(vectorizer.get_feature_names_out())
    
    return f"Основная мысль: {key_phrases}."


def extract_key_thought_method_5(sentences):
    embeddings = get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32)
    centroid = find_centroid(embeddings)
    similarities = cosine_similarity(embeddings, [centroid])
    top_indices = np.argsort(similarities, axis=0)[-3:]  # Выбираем три наиболее близких предложения

    G = nx.Graph()
    for idx in top_indices.flatten():
        G.add_node(sentences[idx])
        G.add_edge('Main Idea', sentences[idx])

    # Конвертируем граф в строковое представление
    concept_map_str = ", ".join([str(node) for node in G.nodes if node != 'Main Idea'])
    return f"Карта концепций: {concept_map_str}"



def dynamic_summarization(text, summarizer, max_length_ratio=0.5, min_length=25, max_increase_step=50):
    input_length = len(text.split())
    # Динамически устанавливаем max_length, чтобы оно не превышало длину входного текста
    max_length = min(int(input_length * max_length_ratio), input_length)
    
    # Увеличиваем max_length, если это необходимо
    while True:
        try:
            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
            return summary[0]['summary_text']
        except ValueError as e:
            if "Input length of input_ids" in str(e):
                max_length += max_increase_step
                if max_length > 800:  # Ограничиваем максимальное увеличение
                    raise ValueError(f"Cannot process the text even after increasing max_length. Current max_length: {max_length}")
            else:
                raise e

def recursive_summarization(text, summarizer, max_length_ratio=0.5, min_length=25, threshold_length=200, max_increase_step=50):
    if len(text.split()) <= threshold_length:
        return dynamic_summarization(text, summarizer, max_length_ratio, min_length, max_increase_step)
    
    sentences = text.split('. ')
    mid_point = len(sentences) // 2
    
    first_half = recursive_summarization('. '.join(sentences[:mid_point]), summarizer, max_length_ratio, min_length, threshold_length, max_increase_step)
    second_half = recursive_summarization('. '.join(sentences[mid_point:]), summarizer, max_length_ratio, min_length, threshold_length, max_increase_step)
    
    combined_summary = first_half + '. ' + second_half
    return dynamic_summarization(combined_summary, summarizer, max_length_ratio, min_length, max_increase_step)

def extract_key_thought_method_3(sentences):
    text = " ".join(sentences)
    return recursive_summarization(text, summarizer, max_length_ratio=0.5, min_length=25, threshold_length=200, max_increase_step=50)


# Применение всех методов к каждому кластеру
def apply_key_thought_extraction(df):
    logger.info(f"extract_key_thought_method_1")
    df['Key_Thought_Method_1'] = df['sentences'].apply(extract_key_thought_method_1)
    logger.info(f"extract_key_thought_method_2")
    df['Key_Thought_Method_2'] = df['sentences'].apply(extract_key_thought_method_2)
    # logger.info(f"extract_key_thought_method_3")
    # df['Key_Thought_Method_3'] = df['sentences'].apply(extract_key_thought_method_3)
    logger.info(f"extract_key_thought_method_4")
    df['Key_Thought_Method_4'] = df['sentences'].apply(extract_key_thought_method_4)
    logger.info(f"extract_key_thought_method_5")
    df['Key_Thought_Method_5'] = df['sentences'].apply(extract_key_thought_method_5)
    return df



Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'GPT2LMHeadModel' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGenera

In [89]:
!export PYTHONIOENCODING=utf-8

In [90]:
from contextlib import contextmanager
@contextmanager
def suppress_output():
    devnull = os.open(os.devnull, os.O_WRONLY)
    old_stdout = os.dup(1)
    old_stderr = os.dup(2)
    os.dup2(old_stdout, 1)
    os.dup2(old_stderr, 2)
    try:
        yield
    finally:
        os.dup2(old_stdout, 1)
        os.dup2(old_stderr, 2)
        os.close(devnull)

In [96]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist
from contextlib import contextmanager
from tqdm import tqdm
import matplotlib.pyplot as plt

def compute_mean_distance(cluster_indices, embeddings):
    # Если в кластере меньше двух элементов, среднее расстояние не вычисляется
    if len(cluster_indices) < 2:
        return 0
    
    # Вычисляем попарные косинусные расстояния между элементами кластера
    cluster_embeddings = embeddings[cluster_indices]
    pairwise_distances = pdist(cluster_embeddings, metric='cosine')
    
    # Возвращаем среднее расстояние
    return np.mean(pairwise_distances)

def cluster_product_reviews(df, model, tokenizer, batch_size=32, max_distance=0.5, min_cluster_size=10):
    grouped = df.groupby('product')
    final_clusters_list = []

    # logger.info("Начало кластеризации продуктов.")

    for product, group in tqdm(grouped, desc="Processing products"):
        # logger.info(f"Обработка продукта: {product}")
        sentences = group["sentences"].tolist()
        embeddings = get_embeddings_batchwise(sentences, model, tokenizer, batch_size=batch_size)
        embeddings = np.array(embeddings, dtype=np.float32)
        
        num_points = len(embeddings)
        # logger.info(f"Количество точек для продукта {product}: {num_points}")

        if num_points < min_cluster_size:
            if num_points == 1:
                initial_clusters = np.array([1] * num_points)
            else:
                initial_clusters = np.array([1] * (num_points // 2) + [2] * (num_points - num_points // 2))
            group['final_cluster'] = initial_clusters
            group['mean_distance'] = 0  # Устанавливаем значение по умолчанию
        else:
            # Иерархическая кластеризация с использованием метода 'ward'
            distance_matrix = pdist(embeddings, metric='cosine')  # Используем косинусное расстояние
            linkage_matrix = linkage(distance_matrix, method='ward')
            initial_clusters = fcluster(linkage_matrix, t=max_distance, criterion='distance')
        
            # logger.info(f"Начальное количество кластеров для продукта {product}: {len(np.unique(initial_clusters))}")

            # Построение дендрограммы для текущего продукта
            # plt.figure(figsize=(10, 7))
            # plt.title(f"Дендрограмма для продукта: {product}")
            # dendrogram(linkage_matrix)
            # plt.show()
        
            # Объединение кластеров до тех пор, пока их размер не будет хотя бы min_size элементов
            final_clusters = initial_clusters
            # logger.info(f"Итоговое количество кластеров для продукта {product}: {len(np.unique(final_clusters))}")
        
            group = group.copy()
            group['final_cluster'] = final_clusters
            
            # Добавляем новый столбец со средним расстоянием внутри кластера
            group['mean_distance'] = group['final_cluster'].apply(lambda cluster_id: compute_mean_distance(np.where(final_clusters == cluster_id)[0], embeddings))
        
        grouped_clusters = group.groupby('final_cluster').agg({
            'category': 'first',
            'product': 'first',
            'review_rating': 'mean',
            'url': 'first',
            'sentences': lambda x: list(x),
            'mean_distance': 'first',  # Добавляем среднее расстояние в итоговый DataFrame
        }).reset_index(drop=True)
                
        grouped_clusters['elem_count'] = grouped_clusters['sentences'].apply(lambda x: len(x))  
        grouped_clusters['word_count'] = grouped_clusters['sentences'].apply(count_words)        
        grouped_clusters['key_thought'] = grouped_clusters['sentences'].apply(extract_key_thought)
        grouped_clusters = grouped_clusters.sort_values(by='word_count', ascending=False)
        
        final_clusters_list.append(grouped_clusters)
    
    # logger.info("Кластеризация завершена.")
    return pd.concat(final_clusters_list, ignore_index=True)

# Применяем кластеризацию ко всему DataFrame
final_df = cluster_product_reviews(df_filtered_reviews, model, tokenizer, batch_size=32, max_distance=0.5, min_cluster_size=5)

# Для вывода финального результата
# logger.info(f"Итоговый DataFrame содержит {len(final_df)} записей.")
final_df.sort_values("word_count", ascending=False)

Processing products:  69%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                  | 11/16 [01:19<00:36,  7.23s/it]


KeyboardInterrupt: 

In [ ]:
final_df[final_df["elem_count"] == 1]

,category,product,review_rating,url,sentences,mean_distance,elem_count,word_count,key_thought
0,/Автотовары/OFFroad,Autobrand_AED / Дополнительная led фара 30w с ...,2.0,https://www.wildberries.ru/catalog/90401367/fe...,"[Но через месяц сгорел один диод, второй ещё ч...",0.0,1,13,"Но через месяц сгорел один диод, второй ещё че..."
7,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.0,https://www.wildberries.ru/catalog/124191551/f...,[В Казахстан Костанай скую область город Лисак...,0.0,1,14,В Казахстан Костанай скую область город Лисако...
8,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.0,https://www.wildberries.ru/catalog/124191551/f...,[Отлично всё пока не установим вскрыть пока не...,0.0,1,13,Отлично всё пока не установим вскрыть пока не ...
9,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.0,https://www.wildberries.ru/catalog/124191551/f...,[Коробка с лебедкой пришла как будто её собаки...,0.0,1,13,Коробка с лебедкой пришла как будто её собаки ...
10,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.0,https://www.wildberries.ru/catalog/124191551/f...,"[Все работает все, устраивает единственное что...",0.0,1,12,"Все работает все, устраивает единственное что ..."
...,...,...,...,...,...,...,...,...,...
1366,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.0,https://www.wildberries.ru/catalog/155565431/f...,[фото],0.0,1,1,фото
1367,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.0,https://www.wildberries.ru/catalog/155565431/f...,[выступала],0.0,1,1,выступала
1368,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.0,https://www.wildberries.ru/catalog/155565431/f...,[были],0.0,1,1,были
1369,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,3.0,https://www.wildberries.ru/catalog/155565431/f...,[ее],0.0,1,1,ее


In [93]:
final_df[final_df["elem_count"] == 1].info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 1205 entries, 0 to 1370
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   category       1205 non-null   object
 1   product        1205 non-null   object
 2   review_rating  1205 non-null   float64
 3   url            1205 non-null   object
 4   sentences      1205 non-null   list
 5   mean_distance  1205 non-null   float64
 6   elem_count     1205 non-null   int64
 7   word_count     1205 non-null   int64
 8   key_thought    1205 non-null   object
dtypes: float64(2), int64(2), list(1), object(4)
memory usage: 377.9+ KB


In [94]:
final_df_filtered = final_df[final_df["elem_count"] > 2]
final_df_filtered.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 59 entries, 6 to 1301
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   category       59 non-null     object
 1   product        59 non-null     object
 2   review_rating  59 non-null     float64
 3   url            59 non-null     object
 4   sentences      59 non-null     list
 5   mean_distance  59 non-null     float64
 6   elem_count     59 non-null     int64
 7   word_count     59 non-null     int64
 8   key_thought    59 non-null     object
dtypes: float64(2), int64(2), list(1), object(4)
memory usage: 20.9+ KB


In [95]:
# Применяем функцию к итоговому DataFrame
final_df_filtered = apply_key_thought_extraction(final_df_filtered)

2024-08-23 11:20:25.582 | INFO     | __main__:apply_key_thought_extraction:102 - extract_key_thought_method_1
2024-08-23 11:20:33.618 | INFO     | __main__:apply_key_thought_extraction:104 - extract_key_thought_method_2
2024-08-23 11:20:47.266 | INFO     | __main__:apply_key_thought_extraction:108 - extract_key_thought_method_4


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
final_df_filtered

,category,product,review_rating,url,sentences,mean_distance,elem_count,word_count,key_thought,Key_Thought_Method_1,Key_Thought_Method_2
2,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,"[Если не топить ее в воде полностью, то влаги ...",0.106140,3,91,Коробка с лебедкой пришла как будто её собаки ...,Коробка с лебедкой пришла как будто её собаки ...,Брад до этого меньшей мощность немного не хват...
3,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,3.333333,https://www.wildberries.ru/catalog/124191551/f...,"[Рычаг свободного хода лебедки не герметичный,...",0.161980,3,40,"Рычаг свободного хода лебедки не герметичный, ...","Рычаг свободного хода лебедки не герметичный, ...","Куда смотрите???заказывали стальной трос, а пр..."
58,/Автотовары/OFFroad,MOTORin / Расширитель колёсных арок 40 мм,3.666667,https://www.wildberries.ru/catalog/149157758/f...,"[На ниву нормально не встаёт, окантовка малень...",0.139602,3,46,"Сами расширители арок нормальные, но ставить н...","Сами расширители арок нормальные, но ставить н...","Для установки требуется частое сверление арок,..."
107,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 3500lb 15...,3.200000,https://www.wildberries.ru/catalog/162316088/f...,[Просто для некоторых товарищей этот прибор до...,0.141455,5,100,"Нормальный лебёдка, но шумел, разобрался полож...","Нормальный лебёдка, но шумел, разобрался полож...","Непонятна работа тормоза, на видео сработал 2 ..."
108,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 3500lb 15...,5.000000,https://www.wildberries.ru/catalog/162316088/f...,"[Хорошая лебедка, со своей задачей справляется...",0.114982,3,43,Лебёдку использую для поднятия и опускания сне...,Лебёдку использую для поднятия и опускания сне...,"Хорошая компактная лебёдка, брал для затаскива..."
131,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 4500lb 20...,3.200000,https://www.wildberries.ru/catalog/240093970/f...,[Просто для некоторых товарищей этот прибор до...,0.141455,5,100,"Нормальный лебёдка, но шумел, разобрался полож...","Нормальный лебёдка, но шумел, разобрался полож...","Непонятна работа тормоза, на видео сработал 2 ..."
132,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 4500lb 20...,5.000000,https://www.wildberries.ru/catalog/240093970/f...,"[Хорошая лебедка, со своей задачей справляется...",0.114982,3,43,Лебёдку использую для поднятия и опускания сне...,Лебёдку использую для поднятия и опускания сне...,"Хорошая компактная лебёдка, брал для затаскива..."
160,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.428571,https://www.wildberries.ru/catalog/48839109/fe...,"[Хороши когда нужно ""выскочить"" из снежного ме...",0.093537,7,144,"Хороши когда нужно ""выскочить"" из снежного мес...","Хороши когда нужно ""выскочить"" из снежного мес...","В снегопады спасают, чтобы выехать с заснеженн..."
161,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.000000,https://www.wildberries.ru/catalog/48839109/fe...,"[Можно было отлив снизу сделать, чтобы не выпа...",0.127342,11,135,Хотел вытащить катер из воды на переднем приво...,Хотел вытащить катер из воды на переднем приво...,"Ещё не пробовал, но я думаю на морозе пластик ..."
162,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.000000,https://www.wildberries.ru/catalog/48839109/fe...,"[Не во всех случаях, конечно, эти Анти буксы м...",0.117228,7,131,Зимняя резина пережевывает эти пластиковые ант...,Зимняя резина пережевывает эти пластиковые ант...,"Было бы чуть лучше, если бы шипы на пластинах ..."


In [ ]:
final_df_filtered

In [ ]:
final_df.sort_values("word_count", ascending=False)

,category,product,review_rating,url,sentences,mean_distance,elem_count,word_count,key_thought
394,/Автотовары/OFFroad,ПК ЛИМ / Браслеты цепи противоскольжения,3.928571,https://www.wildberries.ru/catalog/143132420/f...,[Уже пользовался несколько раз))) к сожалению)...,0.099308,14,315,"Протягивать ленту с той стороны колеса, одурев..."
464,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,3.928571,https://www.wildberries.ru/catalog/143132161/f...,[Уже пользовался несколько раз))) к сожалению)...,0.099308,14,315,"Протягивать ленту с той стороны колеса, одурев..."
304,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,3.571429,https://www.wildberries.ru/catalog/62827233/fe...,[ЗАСТРЯЛА!ДУМАЛА УЖЕ ВЕСНЫ ЖДАТЬ🙈МАШИНА СТЯЛА ...,0.110913,7,198,"Хоть производители пишет, что буксовать нельзя..."
647,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.777778,https://www.wildberries.ru/catalog/155565431/f...,[Реально штука работает намазать на ржавчину с...,0.123763,9,179,"Попробовал на машине ржа до дыры на дверке, по..."
160,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.428571,https://www.wildberries.ru/catalog/48839109/fe...,"[Хороши когда нужно ""выскочить"" из снежного ме...",0.093537,7,144,"Хороши когда нужно ""выскочить"" из снежного мес..."
...,...,...,...,...,...,...,...,...,...
646,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,5.000000,https://www.wildberries.ru/catalog/197410221/f...,[.],0.000000,1,1,.
771,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,[Нанёс],0.000000,1,1,Нанёс
772,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,1.000000,https://www.wildberries.ru/catalog/155565431/f...,[Го.],0.000000,1,1,Го.
773,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,1.000000,https://www.wildberries.ru/catalog/155565431/f...,[Но.],0.000000,1,1,Но.


In [ ]:
final_df.sort_values("product", ascending=False)

,category,product,review_rating,url,sentences,mean_distance,elem_count,word_count,key_thought
775,/Автотовары/OFFroad,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,4.000000,https://www.wildberries.ru/catalog/111627554/f...,[Встали как родные устанавливал на петро пикап...,0.000000,2,19,"На вид не плохие, но к сожалению на уаз профи ..."
776,/Автотовары/OFFroad,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,5.000000,https://www.wildberries.ru/catalog/111627554/f...,"[Пришли качество 🔥пока ещё не ставил, Все вста...",0.000000,2,19,Пришли качество 🔥пока ещё не ставил
647,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.777778,https://www.wildberries.ru/catalog/155565431/f...,[Реально штука работает намазать на ржавчину с...,0.123763,9,179,"Попробовал на машине ржа до дыры на дверке, по..."
648,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.333333,https://www.wildberries.ru/catalog/155565431/f...,"[Сам раствор мне понравился, выступала ржавчин...",0.119070,6,110,"Да и на жаре стояла машина, где были пятна ржа..."
649,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,"[Гель супер, очень порадовал, не стекает, можн...",0.137930,4,74,Прешёл по графику работает супер до этого проб...
...,...,...,...,...,...,...,...,...,...
53,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,[Не проверял],0.000000,1,2,Не проверял
54,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,[Установлю],0.000000,1,1,Установлю
55,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,[be/X6alkqqmbiQ?si=_dV1lFSghhipjst6],0.000000,1,1,be/X6alkqqmbiQ?si=_dV1lFSghhipjst6
0,/Автотовары/OFFroad,Autobrand_AED / Дополнительная led фара 30w с ...,2.000000,https://www.wildberries.ru/catalog/90401367/fe...,"[Но через месяц сгорел один диод, второй ещё ч...",0.000000,1,13,"Но через месяц сгорел один диод, второй ещё че..."


In [ ]:
# Удаление записей с word_count <= 10 и ключевой мыслью менее 3 символов
final_result = final_result[((final_result['word_count'] > 10) & (final_result['key_thought'].str.len() > 5))]
final_result

NameError: name 'final_result' is not defined

In [ ]:
final_result.to_csv("./reviews_keywords/feedbackfueltest.csv")